# [Exploration 06] 멋진 작사가 만들기
## 01. download data
- 학습할 텍스트 데이터를 다운로드합니다.
- 해당 프로젝트에서는 작사가를 만들기 위해 가사 데이터를 다운로드합니다. 

## 02. load data 
- glob 모듈을 사용해 파일을 읽습니다. 
- 문장 단위로 끊어서 corpus(말뭉치 데이터)를 저장합니다. 

In [41]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []


for txt_file in txt_list:
    # with구문으로 txt_list에 있는 txt_file 하나하나를 열어줍니다.
    with open(txt_file, "r") as f:
        # line을 기준으로 문장단위로 끊어서 raw변수에 넣어줍니다.
        raw = f.read().splitlines()
        # 리스트로 생성된 raw는 append가 아닌 extend로 raw_corpus에 더합니다.
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I. LIFE.', '', '']


## 03. preprocess data 
- 적절한 문장의 길이를 구해서, padding을 하거나 잘라줍니다.
- tf.keras.preprocessing.text.Tokenizer 패키지를 이용해 정제된 데이터를 토큰화합니다.
- 토큰을 모아 사전을 만들고, 숫자로 변환해줍니다.
- 이때 숫자로 변환된 데이터(해당 프로젝트에서는 노래 가사)를 텐서(tensor)라고 합니다.
- 해당 프로젝트에서는 한 문장에 토큰 15개를 권장합니다.

In [23]:
raw_corpus[:15]

['I. LIFE.',
 '',
 '',
 '',
 '',
 '',
 'I.',
 '',
 'SUCCESS.',
 '',
 '[Published in "A Masque of Poets"',
 'at the request of "H.H.," the author\'s',
 'fellow-townswoman and friend.]',
 '',
 'Success is counted sweetest']

In [42]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()      
  
    # 불필요한 특수기호, 공백 등의 패턴을 지우고 공백으로 단어를 토큰화할 수 있도록 정제합니다
    sentence = re.sub(r"([?.!,¿])", r" ", sentence)        # 패턴의 특수문자 -> 공백
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴 -> 스페이스 1개
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도) -> 스페이스 1개

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [43]:
sentence = '[Published in "A Masque of Poets"'
sentence_1 = preprocess_sentence(sentence)
sentence_1

'<start> published in a masque of poets <end>'

In [44]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
#     if len(sentence) > 15:
#         sentence = sentence[:15]
        
    corpus.append(preprocess_sentence(sentence))

corpus[:10]

['<start> i life <end>',
 '<start> i <end>',
 '<start> success <end>',
 '<start> published in a masque of poets <end>',
 '<start> at the request of h h the author s <end>',
 '<start> fellow townswoman and friend <end>',
 '<start> success is counted sweetest <end>',
 '<start> by those who ne er succeed <end>',
 '<start> to comprehend a nectar <end>',
 '<start> requires sorest need <end>']

In [30]:
import tensorflow as tf

def tokenize(corpus):
    # Tokenizer 패키지를 생성합니다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장의 크기를 설정합니다. (권장 12,000 이상)
        filters=' ',    # 별도의 전처리 로직
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen는 시퀀스 길이를 뜻합니다. 지정하지 않을 때는 None이 디폴트값입니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,'\n', tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

len(tensor), len(corpus)

[[   2    4  156 ...    0    0    0]
 [   2    4    3 ...    0    0    0]
 [   2 3098    3 ...    0    0    0]
 ...
 [   2    5  832 ...    0    0    0]
 [   2   21   54 ...    0    0    0]
 [   2   51    4 ...    0    0    0]] 


(175986, 175986)

## 04. split data
## 05. build model